In [ ]:
'''
論文爬蟲目標
https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/login?o=dwebmge
'''

In [4]:
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
# from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
# from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
# from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
# import subprocess

# 正規表達式
import re

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                # 不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")           # 最大化視窗
my_options.add_argument("--incognito")                 # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")    # 禁用彈出攔截
my_options.add_argument("--disable-notifications")     # 取消通知
my_options.add_argument("--lang=zh-TW")                # 設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

# 建立儲存圖片、影片的資料夾
folderPath = 'Thesis'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\John1\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [5]:
def visit():
    driver.get('https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/login?o=dwebmge')
    
def search():
    SearchKeys = input('請輸入查詢關鍵字:' )
    txtInput = driver.find_element(By.CSS_SELECTOR, 'input.inputbox')
    txtInput.send_keys(SearchKeys)
    sleep(2)
    kwBox = driver.find_element(By.CSS_SELECTOR, 'input[value="kw"]')
    kwBox.click()
    sleep(2)
    abBox = driver.find_element(By.CSS_SELECTOR, 'input[value="ab"]')
    abBox.click()
    sleep(2)
    txtInput.submit()
    sleep(3)
    
def selectTheses():
    theses = driver.find_elements(By.CSS_SELECTOR, 'td.tdfmt1-content')
    
    for i in range(10):
        reference = theses[i].find_element(By.XPATH, '..')
        resultNum = reference.find_element(By.CSS_SELECTOR, 'td.tdfmt1-second').text
        linkElement = theses[i].find_element(By.CSS_SELECTOR, 'a.slink')
        titleTag = linkElement.find_element(By.CSS_SELECTOR, 'span.etd_d')
        thesisTitle = titleTag.get_attribute('innerText')
        tempLink = linkElement.get_attribute('href')
        
        # 開啟新分頁
        driver.execute_script("window.open('');")
        # 切換到新分頁
        driver.switch_to.window(driver.window_handles[1])
        driver.get(tempLink)
        sleep(3)
        
        # 取得論文永久連結、說明
        permaLink = driver.find_elements(By.CSS_SELECTOR, 'input#fe_text1')[0].get_attribute("value")
        description = driver.find_elements(By.CSS_SELECTOR, 'meta[name="description"]')[0].get_attribute("content")
            
        # 關掉新分頁
        driver.close()
        # 回到原分頁
        driver.switch_to.window(driver.window_handles[0])
        sleep(3)    

        listData.append({
            'resulNum':resultNum,
            'title':thesisTitle,
            'link':permaLink,
            'description':description
        })
        
def saveJson():
    with open(f"{folderPath}/thesis.json", "w", encoding="utf-8" ) as file:
        file.write(json.dumps(listData, ensure_ascii=False))       
        
def close():
    driver.quit()

In [ ]:
# 論文系統翻頁用 - 測試翻頁功能
def turnPage():
    regexTotalPage = r'/([\d])'
    currentPage = driver.find_element(By.CSS_SELECTOR, 'input[name="jmpage"]')
    currentPageValue = int(currentPage.get_attribute('value'))
    totalPageTag = currentPage.find_element(By.XPATH, '../..') # 利用currentPage的位置往回選標籤 (2層父標籤)
    totalPage = int(re.search(regexTotalPage, totalPageTag.text)[1]) # 用regex從totalPageTag裡取出數字強制轉換為int
    while currentPageValue < totalPage:
        currentPage = driver.find_element(By.CSS_SELECTOR, 'input[name="jmpage"]') # 重新選擇頁碼位置
        currentPageValue = int(currentPage.get_attribute('value')) # 取出頁碼數字
        print(f'目前在第 {currentPageValue} 頁') # 印出頁碼數字確認
        currentPage.clear() # 確認頁碼後清空欄
        sleep(1)
        currentPage.send_keys(currentPageValue+1) # 輸入下一個要去的頁面
        sleep(2)
        button = driver.find_element(By.CSS_SELECTOR, 'input[name="jumpfmt1page"]') # 點擊換頁按鈕
        button.click()
        sleep(3)
    print(f'已翻到最後一頁: {currentPageValue}')

In [ ]:
# 論文系統爬蟲翻頁用
def turnPage():
    regexTotalPage = r'/([\d])'
    currentPageTag = driver.find_element(By.CSS_SELECTOR, 'input[name="jmpage"]') # 選擇目前頁碼的input欄位
    currentPage = int(currentPageTag.get_attribute('value')) # 從欄位取得目前頁碼
    totalPageTag = currentPageTag.find_element(By.XPATH, '../..') # 利用currentPageTag的位置往回選總共頁數在的標籤 (回逤2層)
    totalPage = int(re.search(regexTotalPage, totalPageTag.text)[1]) # 從totalPageTag取出總共頁數
    if currentPage == totalPage: # 如果目前頁碼 = 總共頁數代表已爬到最後一頁，此時跳脫
        return False
    else:
        currentPageTag.clear() # 清空欄輸入欄以利再次輸入
        sleep(1)
        currentPageTag.send_keys(currentPage+1) # 輸入下一個要去的頁面
        sleep(2)
        button = driver.find_element(By.CSS_SELECTOR, 'input[name="jumpfmt1page"]') # 點擊換頁按鈕
        button.click()
        sleep(3)
        return True

In [ ]:
# 選擇論文標題、連結 - 全部選擇
def selectTheses():
    theses = driver.find_elements(By.CSS_SELECTOR, 'td.tdfmt1-content')
    
    for thesis in theses:
        a = thesis.find_element(By.CSS_SELECTOR, 'a.slink')
        titleTag = a.find_element(By.CSS_SELECTOR, 'span.etd_d')
        thesisTitle = titleTag.get_attribute('innerText')
        tempLink = a.get_attribute('href')
        
        # 開啟新分頁
        driver.execute_script("window.open('');")
        # 切換到新分頁
        driver.switch_to.window(driver.window_handles[1])
        driver.get(tempLink)
        sleep(3)
        
        # 取得論文永久連結
        permaLink = driver.find_elements(By.CSS_SELECTOR, 'input#fe_text1')[0].get_attribute("value")
        description = driver.find_elements(By.CSS_SELECTOR, 'meta[name="description"]')[0].get_attribute("content")
            
        # 關掉新分頁
        driver.close()
        # 回到原分頁
        driver.switch_to.window(driver.window_handles[0])
        sleep(3)    

        listData.append({
            'title':thesisTitle,
            'link':permaLink,
            'description':description
        })

In [15]:
# 主程式 - 只爬第一頁
if __name__ == "__main__":
    visit()
    search()
    selectTheses()
    saveJson()

請輸入查詢關鍵字: 大數據


In [ ]:
# 主程式 - 測試翻頁功能用
if __name__ == "__main__":
    visit()
    search()
    turnPage()

In [ ]:
# 主程式 - 加上翻頁爬蟲功能
if __name__ == "__main__":
    hasNext = True
    visit()
    search()
    while hasNext == True:
        selectTheses()
        hasNext = turnPage()
    saveJson()

In [6]:
# 主程式 - 測試搜尋功能用
if __name__ == "__main__":
    visit()
    search()

請輸入查詢關鍵字: 天氣 發電


In [7]:
close()

Selenium在新tab中開啟網頁/關閉網頁:
https://justcode.ikeepstudying.com/2019/09/python-selenium%E5%A4%9A%E7%AA%97%E5%8F%A3%E4%B9%8B%E5%89%8D%E7%9A%84%E5%88%87%E6%8D%A2-webdriver%E7%AA%97%E5%8F%A3%E5%88%87%E6%8D%A2/

https://www.lambdatest.com/blog/python-selenium-switch-tabs/

從文具區塊copy文字:
https://stackoverflow.com/questions/40980559/copy-text-area-using-selenium-python

從子元素找到父元素:
https://www.tutorialspoint.com/how-to-find-parent-elements-by-python-webdriver

透過selenium清空文字欄位:
https://www.tutorialspoint.com/how-to-clear-the-text-entered-in-selenium-with-python